In [1]:
import dicom
import os
import array

import matplotlib.pyplot as plt

import numpy as np
import SimpleITK as sitk
from ipywidgets import interact
%matplotlib inline



In [3]:
rp=dicom.read_file('/media/sf_CRCT/JupyterNotebook/data/CT/clinic/calmip_test/dicom/RP.201610113.AILEDENEZDTE1.dcm')

In [5]:
iso=[]
for val in rp.BeamSequence[0].ControlPointSequence[0].IsocenterPosition: iso.append(float(val))
study=str(rp.PatientName.split('^')[0])
print study
print iso


BAKER
[-24.928512979423, -97.707406046065, -60.969761688316]


In [6]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.gray)
    interact(plot_image, myslice=(0, im.shape[0]))
    return; 
def ct_crop(pix, x1crop, x2cropt, y1crop, y2cropt):
    print 'ORIGINAL : '
    ct_viewer(pix)
    x2crop=x2cropt-pix.shape[2]
    y2crop=y2cropt-pix.shape[1]
    pix_crop=pixel_array[:, y1crop:y2crop, x1crop:x2crop]
    print 'CROPPED :'
    ct_viewer(pix_crop)
    return pix_crop;

In [8]:
#read dcm directory and compte ONE sitk 3D image object
reader=sitk.ImageSeriesReader()
#dicom_name=reader.GetGDCMSeriesFileNames('scanX/')
dicom_name=reader.GetGDCMSeriesFileNames('/media/sf_CRCT/JupyterNotebook/data/CT/clinic/calmip_test/dicom/ct/')
reader.SetFileNames(dicom_name)
ct_image = reader.Execute()

print ct_image.GetOrigin()
print ct_image.GetDirection()
dicom_ct_dimension=ct_image.GetSize()
dicom_ct_pixel_size=ct_image.GetSpacing()
print'Dimesions : ' , dicom_ct_dimension
print'Pixels Size :' , dicom_ct_pixel_size

pixel_array=sitk.GetArrayFromImage(ct_image)   #Get numpy n-d array from sitk object to plot it                          
ct_viewer(pixel_array)

(-250.0, -250.0, -188.75)
(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
Dimesions :  (512, 512, 202)
Pixels Size : (0.976562, 0.976562, 1.25)


interactive(children=(IntSlider(value=101, description=u'myslice', max=202), Output()), _dom_classes=('widget-interact',))

In [10]:
# write raw original ct
#pixel_array=pixel_array[::-1,::,:]
#sitk.WriteImage(ct_image, study+'/images/'+study+'_original.mhd')
sitk.WriteImage(ct_image, '/media/sf_CRCT/JupyterNotebook/data/CT/clinic/calmip_test//images/'+study+'_original.mhd')

In [12]:
#select apropriates value to crop scan 
#use sliders to view all the slice and refer to original ct
x1=80  #   X axis lower value (on original ct)
x2=430 #   X axis upper value (on original ct)
y1=120 #   Y axis lower value (on original ct)
y2=400 #   Y axis upper value (on original ct)
pixel_array_crop=ct_crop(pixel_array, x1, x2, y1, y2);

ORIGINAL : 


interactive(children=(IntSlider(value=101, description=u'myslice', max=202), Output()), _dom_classes=('widget-interact',))

CROPPED :


interactive(children=(IntSlider(value=101, description=u'myslice', max=202), Output()), _dom_classes=('widget-interact',))

In [14]:
ct_viewer(pixel_array_crop)

ct_crop=sitk.GetImageFromArray(pixel_array_crop)

print ct_crop.GetSize()

crop_val=[x1, y1, 0]
new_origin=[]

for val1, cropval, space in zip(ct_image.GetOrigin(),crop_val , ct_image.GetSpacing()):
    new_origin.append(val1+(cropval*space))

#for gate simulation frame
new_origin[1]=new_origin[1]*-1
new_origin[2]=new_origin[2]*-1

print(new_origin)

ct_crop.SetOrigin(new_origin)
ct_crop.SetSpacing(ct_image.GetSpacing())

sitk.WriteImage(ct_crop, '/media/sf_CRCT/JupyterNotebook/data/CT/clinic/calmip_test/images/'+study+'.mhd')

interactive(children=(IntSlider(value=101, description=u'myslice', max=202), Output()), _dom_classes=('widget-interact',))

(350, 280, 202)
[-171.87504, 132.81256, 188.75]
